## Importing Libraries

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
warnings.filterwarnings('ignore')

from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.utils import class_weight
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
# from feature_engine.selection import DropCorrelatedFeatures
# from feature_engine.selection import SelectBySingleFeaturePerformance, SelectByShuffling, RecursiveFeatureElimination, RecursiveFeatureAddition, DropConstantFeatures

import lightgbm as lgb
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import CategoricalNB
from pycaret.classification import *

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
from sklearn.metrics import roc_auc_score, confusion_matrix, accuracy_score, classification_report, f1_score

import pickle
import os
import glob
import joblib


In [4]:
pip install feature_engine

Note: you may need to restart the kernel to use updated packages.


In [123]:
pip install imblearn

Note: you may need to restart the kernel to use updated packages.


In [124]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [125]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


## Importing Data

In [5]:
i = 1
frames = []
for name in os.listdir('/home/pankaj_v/Documents/Data_analysis_dixit/dATA/client_data'):
    extents = os.path.splitext('/home/pankaj_v/Documents/Data_analysis_dixit/dATA/client_data'+'/{}'.format(name))[1].lower()
    if extents == '.csv':
        df= pd.read_csv('/home/pankaj_v/Documents/Data_analysis_dixit/dATA/client_data/{}'.format(name))
        frames.append(df)
        print('csv',df.shape)
    elif extents == '.xlsx':
        df= pd.read_excel('/home/pankaj_v/Documents/Data_analysis_dixit/dATA/client_data/{}'.format(name))
        df.columns = df.columns.astype(str)
        frames.append(df)
        print('xlx',df.shape)

result =  pd.concat(frames, axis=0, ignore_index= True)

csv (180, 1060)
csv (244, 1060)
csv (24, 1060)
csv (200, 1060)
csv (302, 1060)
csv (16, 1060)
csv (72, 1060)
csv (244, 1060)
csv (48, 1060)
csv (90, 1060)
csv (314, 1060)
csv (78, 1060)
csv (191, 1060)
csv (310, 1060)
csv (294, 1060)
csv (132, 1059)
csv (224, 1060)
csv (224, 1060)
csv (18, 1060)
csv (236, 1060)
csv (160, 1060)
csv (42, 1060)
csv (246, 1060)
csv (302, 1060)
csv (262, 1060)
csv (162, 1060)
csv (142, 1060)
csv (154, 1060)
csv (256, 1060)
csv (192, 1060)
csv (252, 1060)
csv (78, 1060)
csv (270, 1060)
csv (186, 1060)
csv (29, 1060)
csv (276, 1060)
csv (120, 1060)
csv (80, 1060)
csv (28, 1060)
csv (249, 1060)
csv (120, 1060)
csv (248, 1060)
csv (348, 1060)
csv (290, 1060)
csv (102, 1060)
csv (180, 1060)
csv (110, 1060)
csv (198, 1060)
csv (132, 1060)
csv (222, 1060)
csv (77, 1060)
csv (110, 1060)
csv (10, 1060)
csv (60, 1060)
csv (254, 1060)
csv (36, 1060)
xlx (680, 1059)
csv (210, 1060)
csv (326, 1060)
csv (90, 1060)
csv (180, 1060)
csv (230, 1060)
csv (250, 1060)
csv (302,

In [4]:
result.head()

,,L*,a*,b*,X,Y,Z,Dominant Wavelenght,Whiteness,Purity,...,979.335,980.054,980.772,981.491,982.209,982.928,983.646,984.364,985.082,Unnamed: 1059
0,W_Calibration,100.0000,0.000000,0.00000,95.0422,100.0000,108.8760,565.016,99.9909,0.000017,...,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.00,NaN
1,F_Calibration,100.0000,0.000000,0.00000,108.1040,100.0000,39.2965,491.070,99.9909,0.000170,...,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.00,NaN
2,W_Calibration,100.0000,0.000000,0.00000,95.0422,100.0000,108.8760,565.016,99.9909,0.000017,...,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.00,NaN
3,F_Calibration,100.0000,0.000000,0.00000,108.1040,100.0000,39.2965,491.070,99.9909,0.000170,...,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.00,NaN
4,W_3566A2/33_H_Faint_0.30_SI1,99.7016,-0.840692,1.97141,93.8345,99.2302,104.8660,569.412,90.5883,0.017551,...,97.4079,97.6368,98.0312,97.8384,97.8231,-33.8442,-71.7545,-33.0428,-1563.49,NaN


## Data PrePrpcessing

In [5]:
result.rename(columns = {' ': 'Output'}, inplace=True)

In [6]:
result_3 =result.loc[:,'Output']
result_2= result.loc[:,'300.377':'700.577']
result_1= result.iloc[:,1:14]


In [7]:
df = pd.concat([result_2, result_3], axis=1)

In [8]:
df.shape

(15162, 524)

In [11]:
# df.to_excel('/home/pankaj_v/Documents/Data_analysis_dixit/dATA/Processed_data/whole_data.xlsx')

In [9]:
df_without_calb = df[~df['Output'].str.contains(('Calibration|CALIBARTION|CALIBARATION'),case=False, regex=True, na = False)]
df_without_calb.reset_index(drop=True)
df_without_calb['w/f']= df_without_calb['Output'].str.split('_').str[0]

In [10]:
df_with_w = df_without_calb[df_without_calb['w/f'] == 'W']

In [11]:
df_with_w.size.astype(str)

'2629200'

In [12]:
df_with_w['quality'] = df_with_w['Output'].str.split('_').str[3]
df_with_w['quality'].replace({'lb':'Faint','Benth':'None','brown':'None','Pink':'None', 'LB':'None','LB ':'None', 'Faint YELLOW':'Faint', 'Faint YELLOW':'Faint', 'Faint GREEN':'Faint','  ':'None', 'BROWN':'Faint', 'Lblack':'Faint', 'lb ':'None','Black':'None'}, inplace = True)

In [13]:
df_with_w['color'] = df_with_w['Output'].str.split('_').str[2]
df_with_w['color'].replace({'d':'D','H/BLK':'H','E/LB':'E','H/LB':'H','J/LB':'J','I/LB':'I','J ':'J','F/LB':'F','G/LB':'G','E LB':'E', ' K':'K', 'G LB ':'G', 'H ':'H'}, inplace = True)


In [14]:
df_with_w.drop((df_with_w[df_with_w.color.str.contains('\s', regex=True)].index|df_with_w[df_with_w['color'] == ''].index|df_with_w[df_with_w['quality'].str.contains('\s\W', regex=True)].index|df_with_w[df_with_w['color'].str.contains('\s\W', regex=True)].index|df_with_w[df_with_w['quality'].str.contains(('^\s'), regex=True)].index), inplace=True)

In [15]:
df_with_w.drop(columns=['Output','w/f'], inplace=True)

In [16]:
df_with_w = df_with_w[~(df_with_w['quality'] == 'Strong')]

In [17]:
df_with_w.to_csv('/home/pankaj_v/Documents/Data_analysis_dixit/dATA/Processed_data/df_w_1_08.csv')

In [35]:
df_train = df_with_w.copy()
df_train.reset_index(drop=True, inplace=True)

In [46]:
df_train['300.377'].astype('float').min()

65.1567

## Reomoving Outliers

In [45]:
# df_train.drop(index = df_train['300.377'].astype('float').idxmax(), inplace=True)
df_train.drop(index=df_train['300.377'].astype('float').idxmin(), inplace=True)

In [48]:
df_train.reset_index(drop=True, inplace=True)

In [50]:
df_train.to_csv('/home/pankaj_v/Documents/Data_analysis_dixit/dATA/Processed_data/df_train_2_08.csv')

In [16]:
df_train = pd.read_csv('/home/pankaj_v/Documents/Data_analysis_dixit/dATA/Processed_data/df_train_2_08.csv')

In [17]:
df_train.drop(columns='Unnamed: 0', inplace=True)

In [34]:
df_train

,300.377,301.162,301.946,302.731,303.515,304.299,305.083,305.867,306.652,307.435,308.219,309.003,309.787,310.571,311.354,312.138,312.921,313.705,314.488,315.272,316.055,316.838,317.621,318.404,319.187,319.97,320.753,321.536,322.318,323.101,323.884,324.666,325.449,326.231,327.013,327.795,328.578,329.36,330.142,330.924,331.706,332.488,333.269,334.051,334.833,335.614,336.396,337.177,337.959,338.74,339.521,340.303,341.084,341.865,342.646,343.427,344.207,344.988,345.769,346.55,347.33,348.111,348.891,349.672,350.452,351.232,352.012,352.793,353.573,354.353,355.133,355.912,356.692,357.472,358.252,359.031,359.811,360.59,361.37,362.149,362.928,363.707,364.486,365.266,366.044,366.823,367.602,368.381,369.16,369.938,370.717,371.496,372.274,373.052,373.831,374.609,375.387,376.165,376.943,377.721,378.499,379.277,380.055,380.833,381.61,382.388,383.165,383.943,384.72,385.498,386.275,387.052,387.829,388.606,389.383,390.16,390.937,391.714,392.49,393.267,394.044,394.82,395.596,396.373,397.149,397.925,398.702,399.478,400.254,401.03,401.806,402.581,403.357,404.133,404.908,405.684,406.459,407.235,408.01,408.786,409.561,410.336,411.111,411.886,412.661,413.436,414.211,414.985,415.76,416.535,417.309,418.084,418.858,419.632,420.407,421.181,421.955,422.729,423.503,424.277,425.051,425.825,426.598,427.372,428.146,428.919,429.693,430.466,431.239,432.013,432.786,433.559,434.332,435.105,435.878,436.651,437.423,438.196,438.969,439.741,440.514,441.286,442.058,442.831,443.603,444.375,445.147,445.919,446.691,447.463,448.235,449.007,449.778,450.55,451.321,452.093,452.864,453.636,454.407,455.178,455.949,456.72,457.491,458.262,459.033,459.804,460.574,461.345,462.116,462.886,463.656,464.427,465.197,465.967,466.738,467.508,468.278,469.048,469.817,470.587,471.357,472.127,472.896,473.666,474.435,475.205,475.974,476.743,477.512,478.281,479.051,479.82,480.588,481.357,482.126,482.895,483.663,484.432,485.2,485.969,486.737,487.505,488.274,489.042,489.81,490.578,491.346,492.114,492.881,493.649,494.417,495.184,495.952,496.719,497.487,498.254,499.021,499.788,500.556,quality,color
0,93.9613,93.5905,93.5144,93.5858,94.0028,93.9788,94.2287,93.9844,94.2687,94.7686,95.0092,95.0911,94.8628,94.4701,94.3056,94.1119,94.4940,94.1478,94.1163,94.1661,94.2465,94.3433,94.4128,94.0820,94.3857,94.1324,94.6203,94.4565,94.3913,94.3538,94.3034,94.3635,94.4522,94.1456,94.1879,94.0829,94.1186,94.2287,94.1389,94.1498,94.3236,94.5980,94.5696,94.6827,94.4149,94.3401,94.2201,94.0791,94.0725,93.6401,93.1323,93.1285,93.1028,92.8175,92.7274,92.4290,92.3611,92.0071,91.8828,91.9737,92.0631,91.6995,91.6406,91.6824,91.6856,91.7052,91.6711,91.7655,91.5221,91.3002,91.3395,90.9359,90.7462,90.5828,90.3397,90.3154,90.0729,90.0673,89.8336,89.4694,89.6399,89.4155,89.2173,89.1439,89.0943,88.8824,88.8579,88.7825,88.7835,88.5097,88.4302,88.5361,88.3171,88.2024,88.2092,88.0088,87.9187,87.8081,87.7571,87.5274,87.2819,87.1189,86.9690,86.7501,86.4363,86.0908,85.7997,85.5023,85.2375,84.9186,84.4828,83.9220,83.4471,82.9483,82.4136,81.7960,81.1918,80.4758,79.7278,79.0685,78.5167,77.8794,77.2856,76.6810,76.0253,75.2729,74.5040,73.7853,73.0899,72.5984,72.2843,72.2457,72.4610,72.8500,73.3974,74.0383,74.8120,75.6796,76.6133,77.6116,78.4938,79.1364,79.4559,79.5641,79.9748,80.6503,81.5699,82.5066,83.4673,84.5349,85.8319,87.4785,89.3028,90.7707,91.8796,92.8069,93.5117,94.1811,94.7576,95.2290,95.6169,95.9703,96.1846,96.3193,96.3532,96.3907,96.4601,96.4965,96.5405,96.5335,96.5143,96.5632,96.5968,96.6472,96.7089,96.7113,96.7318,96.7283,96.7243,96.7127,96.7120,96.7092,96.6860,96.6435,96.6407,96.6093,96.5683,96.5567,96.5310,96.4586,96.4315,96.3958,96.3904,96.3910,96.3819,96.3894,96.4129,96.4498,96.5384,96.5586,96.6312,96.6823,96.7116,96.7726,96.8312,96.8677,96.9374,96.9516,97.0588,97.1030,97.1709,97.2152,97.2793,97.3319,97.3598,97.3777,97.3986,97.3852,97.3667,97.3374,97.2905,97.1929,97.0941,97.0100,96.9223,96.8944,96.8647,96.8855,96.9661,97.0889,97.2527,97.4735,97.6894,97.8879,98.0954,98.2602,98.4187,98.5324,98.6259,98.7014,98.7576,98.82

# Model Implementation

In [28]:
from pycaret.classification import *

In [49]:
reg = setup(data = df_train,fix_imbalance = True, normalize = True,normalize_method = 'zscore',data_split_stratify = True, target = 'color', remove_perfect_collinearity = False)

,Description,Value
0,session_id,3748
1,Target,color
2,Target Type,Multiclass
3,Label Encoded,"D: 0, E: 1, F: 2, G: 3, H: 4, I: 5, J: 6, K: 7, L: 8, M: 9"
4,Original Data,"(4998, 261)"
5,Missing Values,False
6,Numeric Features,259
7,Categorical Features,1
8,Ordinal Features,False
9,High Cardinality Features,False


In [61]:
compare_models(include=['lr','et','rf','xgboost','ada'])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.4614,0.8627,0.4120,0.4700,0.4628,0.3841,0.3847,1.1390
et,Extra Trees Classifier,0.4568,0.8658,0.4031,0.4629,0.4570,0.3782,0.3788,0.4780
xgboost,Extreme Gradient Boosting,0.4508,0.8634,0.3922,0.4578,0.4514,0.3714,0.3720,50.7300
lr,Logistic Regression,0.3728,0.8189,0.3463,0.3753,0.3595,0.2854,0.2886,1.7000
ada,Ada Boost Classifier,0.2156,0.6489,0.2202,0.2080,0.1574,0.1083,0.1233,2.0230


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=3748, verbose=0,
                       warm_start=False)

In [43]:
extra_trees=create_model('et')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5057,0.8763,0.4469,0.5095,0.5045,0.4303,0.4311
1,0.4686,0.8650,0.3769,0.4630,0.4628,0.3886,0.3893
2,0.4829,0.8733,0.3846,0.4641,0.4694,0.4042,0.4052
3,0.4657,0.8594,0.3708,0.4612,0.4618,0.3857,0.3862
4,0.4829,0.8773,0.3852,0.4720,0.4755,0.4047,0.4054
5,0.5086,0.8709,0.3959,0.4990,0.4973,0.4339,0.4359
6,0.4829,0.8717,0.4035,0.4740,0.4770,0.4041,0.4045
7,0.4743,0.8725,0.3814,0.4664,0.4688,0.3957,0.3961
8,0.4756,0.8599,0.3782,0.4692,0.4685,0.3965,0.3977


In [42]:
log = create_model('lr')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5171,0.8954,0.4021,0.4998,0.5002,0.4423,0.4446
1,0.5286,0.8837,0.4219,0.5042,0.5076,0.4554,0.4584
2,0.5400,0.8935,0.4301,0.5259,0.5251,0.4693,0.4718
3,0.5114,0.8978,0.3967,0.4937,0.4929,0.4348,0.4378
4,0.5086,0.8987,0.4021,0.4796,0.4819,0.4314,0.4349
5,0.5257,0.8836,0.4127,0.5019,0.5076,0.4524,0.4540
6,0.5029,0.8874,0.3944,0.4839,0.4887,0.4255,0.4269
7,0.5114,0.8882,0.4076,0.4937,0.4878,0.4354,0.4396
8,0.5387,0.9029,0.4218,0.5252,0.5237,0.4670,0.4695


In [44]:
tuned_extra_classifier=tune_model(extra_trees)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.4743,0.8691,0.4662,0.4868,0.4786,0.3987,0.3991
1,0.4257,0.8543,0.4579,0.4370,0.4274,0.3442,0.3449
2,0.4143,0.8621,0.3367,0.4131,0.4082,0.3302,0.3312
3,0.4057,0.8468,0.3506,0.4074,0.4045,0.3206,0.3210
4,0.4886,0.8819,0.4878,0.4988,0.4901,0.4164,0.4172
5,0.4257,0.8610,0.3513,0.4372,0.4261,0.3442,0.3455
6,0.4743,0.8775,0.4670,0.4836,0.4767,0.3987,0.3992
7,0.4286,0.8631,0.3793,0.4381,0.4316,0.3473,0.3477
8,0.4499,0.8506,0.3842,0.4639,0.4519,0.3711,0.3725


In [45]:
tuned_log_classifier=tune_model(log)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5571,0.8997,0.4343,0.5440,0.5418,0.4883,0.4912
1,0.5229,0.8853,0.4196,0.5120,0.5118,0.4503,0.4518
2,0.5343,0.8907,0.4222,0.5200,0.5171,0.4621,0.4648
3,0.5029,0.8958,0.3870,0.4849,0.4799,0.4240,0.4282
4,0.5286,0.9028,0.4197,0.5062,0.5074,0.4554,0.4583
5,0.5200,0.8878,0.4112,0.5112,0.5128,0.4478,0.4485
6,0.4943,0.8871,0.3960,0.4790,0.4849,0.4167,0.4172
7,0.5343,0.8898,0.4203,0.5178,0.5154,0.4629,0.4660
8,0.5559,0.9054,0.4380,0.5360,0.5355,0.4866,0.4897


In [46]:
tuned_extra_classifier

ExtraTreesClassifier(bootstrap=True, ccp_alpha=0.0,
                     class_weight='balanced_subsample', criterion='entropy',
                     max_depth=11, max_features='sqrt', max_leaf_nodes=None,
                     max_samples=None, min_impurity_decrease=0.0002,
                     min_impurity_split=None, min_samples_leaf=5,
                     min_samples_split=5, min_weight_fraction_leaf=0.0,
                     n_estimators=290, n_jobs=-1, oob_score=False,
                     random_state=2144, verbose=0, warm_start=False)

### Train Test Split

In [112]:
X = df_X_train

In [114]:
Y = df_train.iloc[:,-1]

In [115]:
X.shape, Y.shape

((4993, 527), (4993,))

In [116]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state =42, stratify = Y)

In [130]:
X_train_cv, X_test_cv, Y_train_cv, Y_test_cv = train_test_split(X_train,Y_train, test_size = 0.2, stratify = Y_train)

In [118]:
balanced_ = class_weight.compute_class_weight(class_weight='balanced', classes=Y.value_counts().keys(),y=Y)
balanced = dict(zip(df_train['color'].value_counts().keys(),balanced_))
balanced

{'D': 0.6171817058096415,
 'G': 0.6187112763320942,
 'F': 0.682103825136612,
 'E': 0.823927392739274,
 'H': 0.8491496598639455,
 'I': 0.9565134099616859,
 'J': 1.0358921161825727,
 'K': 1.4265714285714286,
 'L': 7.8015625,
 'M': 15.13030303030303}

In [131]:
et = ExtraTreesClassifier(criterion = 'entropy',
                            class_weight=balanced, 
                            warm_start=True, 
                            max_samples=None, 
                            bootstrap=True,
                            max_depth=11,
                            n_estimators=1000
                            )

for loop imple

In [133]:
for i in range(10):
    X_train_cv, X_test_cv, Y_train_cv, Y_test_cv = train_test_split(X_train,Y_train, test_size = 0.2, stratify = Y_train)
    et.fit(X_train_cv, Y_train_cv)
    y_pred_cv = et.predict(X_test_cv)

    print(f"for {i} th iteration training accuracy {et.score(X_train_cv, Y_train_cv)}")
    print (f"f1_score weighted for {i} th{metrics.f1_score(y_pred=y_pred_cv, y_true=Y_test_cv, average='weighted')}")

for 0 th iteration training accuracy 0.8338028169014085
f1_score weighted for 0 th0.8458400089737739
for 1 th iteration training accuracy 0.8300469483568075
f1_score weighted for 1 th0.8619736551052164
for 2 th iteration training accuracy 0.8347417840375587
f1_score weighted for 2 th0.843662872217313
for 3 th iteration training accuracy 0.8369327073552426
f1_score weighted for 3 th0.8343141680363946
for 4 th iteration training accuracy 0.8303599374021909
f1_score weighted for 4 th0.8616089094981054
for 5 th iteration training accuracy 0.8325508607198748
f1_score weighted for 5 th0.8524683285075617
for 6 th iteration training accuracy 0.8400625978090767
f1_score weighted for 6 th0.8222047315494224
for 7 th iteration training accuracy 0.8406885758998435
f1_score weighted for 7 th0.8199009119154039
for 8 th iteration training accuracy 0.8353677621283255
f1_score weighted for 8 th0.8405925231216643
for 9 th iteration training accuracy 0.8391236306729265
f1_score weighted for 9 th0.82558203

In [134]:
y_test_preds = et.predict(X_test)
print(classification_report(y_true=Y_test, y_pred=y_test_preds))

              precision    recall  f1-score   support

           D       0.73      0.79      0.76       162
           E       0.48      0.44      0.46       121
           F       0.47      0.47      0.47       146
           G       0.40      0.38      0.39       162
           H       0.40      0.42      0.41       118
           I       0.31      0.28      0.29       104
           J       0.46      0.45      0.45        96
           K       0.55      0.70      0.62        70
           L       0.17      0.08      0.11        13
           M       0.20      0.14      0.17         7

    accuracy                           0.48       999
   macro avg       0.42      0.41      0.41       999
weighted avg       0.47      0.48      0.48       999



In [69]:
et.fit(X_train_cv, Y_train_cv)

ExtraTreesClassifier(bootstrap=True,
                     class_weight={'D': 0.6171817058096415,
                                   'E': 0.823927392739274,
                                   'F': 0.682103825136612,
                                   'G': 0.6187112763320942,
                                   'H': 0.8491496598639455,
                                   'I': 0.9565134099616859,
                                   'J': 1.0358921161825727,
                                   'K': 1.4265714285714286, 'L': 7.8015625,
                                   'M': 15.13030303030303},
                     criterion='entropy', max_depth=15, n_estimators=1000,
                     warm_start=True)

In [70]:
# Training accuracy
y_pred_cv = et.predict(X_train_cv)
print(classification_report(y_true= Y_train_cv, y_pred=y_pred_cv))

              precision    recall  f1-score   support

           D       1.00      1.00      1.00       517
           E       1.00      1.00      1.00       388
           F       0.99      1.00      1.00       469
           G       1.00      1.00      1.00       516
           H       1.00      1.00      1.00       376
           I       1.00      1.00      1.00       334
           J       1.00      1.00      1.00       309
           K       1.00      1.00      1.00       224
           L       1.00      1.00      1.00        41
           M       1.00      1.00      1.00        21

    accuracy                           1.00      3195
   macro avg       1.00      1.00      1.00      3195
weighted avg       1.00      1.00      1.00      3195



In [388]:
#  testing Accureacy
y_pred_cv = et.predict(X_test_cv)
print(classification_report(y_true= Y_test_cv, y_pred=y_pred_cv))

              precision    recall  f1-score   support

           D       0.79      0.86      0.83       130
           E       0.63      0.48      0.55        97
           F       0.49      0.42      0.45       117
           G       0.45      0.50      0.47       129
           H       0.39      0.41      0.40        94
           I       0.40      0.40      0.40        84
           J       0.46      0.42      0.44        77
           K       0.50      0.73      0.59        56
           L       0.00      0.00      0.00        10
           M       0.67      0.40      0.50         5

    accuracy                           0.53       799
   macro avg       0.48      0.46      0.46       799
weighted avg       0.52      0.53      0.52       799



In [390]:
et.fit(X_train_cv, Y_train_cv)

ExtraTreesClassifier(bootstrap=True,
                     class_weight={'D': 0.6171817058096415,
                                   'E': 0.823927392739274,
                                   'F': 0.682103825136612,
                                   'G': 0.6187112763320942,
                                   'H': 0.8491496598639455,
                                   'I': 0.9565134099616859,
                                   'J': 1.0358921161825727,
                                   'K': 1.4265714285714286, 'L': 7.8015625,
                                   'M': 15.13030303030303},
                     criterion='entropy', max_depth=15, n_estimators=1000,
                     warm_start=True)

In [391]:
# Training accuracy
y_pred_cv = et.predict(X_train_cv)
print(classification_report(y_true= Y_train_cv, y_pred=y_pred_cv))

              precision    recall  f1-score   support

           D       0.95      0.97      0.96       517
           E       0.94      0.90      0.92       388
           F       0.91      0.88      0.89       469
           G       0.87      0.90      0.89       516
           H       0.87      0.88      0.87       376
           I       0.88      0.88      0.88       334
           J       0.90      0.87      0.88       309
           K       0.85      0.95      0.90       224
           L       0.97      0.76      0.85        41
           M       0.95      0.95      0.95        21

    accuracy                           0.90      3195
   macro avg       0.91      0.89      0.90      3195
weighted avg       0.90      0.90      0.90      3195



In [392]:
#  testing Accureacy
y_pred_cv = et.predict(X_test_cv)
print(classification_report(y_true= Y_test_cv, y_pred=y_pred_cv))

              precision    recall  f1-score   support

           D       0.97      0.97      0.97       130
           E       0.94      0.90      0.92        97
           F       0.90      0.90      0.90       117
           G       0.90      0.88      0.89       129
           H       0.87      0.91      0.89        94
           I       0.87      0.87      0.87        84
           J       0.90      0.94      0.92        77
           K       0.91      0.95      0.93        56
           L       1.00      1.00      1.00        10
           M       1.00      0.60      0.75         5

    accuracy                           0.91       799
   macro avg       0.93      0.89      0.90       799
weighted avg       0.91      0.91      0.91       799



In [394]:
et.fit(X_train_cv, Y_train_cv)

ExtraTreesClassifier(bootstrap=True,
                     class_weight={'D': 0.6171817058096415,
                                   'E': 0.823927392739274,
                                   'F': 0.682103825136612,
                                   'G': 0.6187112763320942,
                                   'H': 0.8491496598639455,
                                   'I': 0.9565134099616859,
                                   'J': 1.0358921161825727,
                                   'K': 1.4265714285714286, 'L': 7.8015625,
                                   'M': 15.13030303030303},
                     criterion='entropy', max_depth=15, n_estimators=1000,
                     warm_start=True)

In [395]:
# Training accuracy
y_pred_cv = et.predict(X_train_cv)
print(classification_report(y_true= Y_train_cv, y_pred=y_pred_cv))

              precision    recall  f1-score   support

           D       0.96      0.97      0.97       517
           E       0.94      0.90      0.92       388
           F       0.91      0.88      0.90       469
           G       0.88      0.89      0.89       516
           H       0.87      0.88      0.88       376
           I       0.86      0.88      0.87       334
           J       0.89      0.87      0.88       309
           K       0.86      0.94      0.90       224
           L       0.97      0.78      0.86        41
           M       1.00      0.90      0.95        21

    accuracy                           0.90      3195
   macro avg       0.91      0.89      0.90      3195
weighted avg       0.90      0.90      0.90      3195



In [396]:
#  testing Accureacy
y_pred_cv = et.predict(X_test_cv)
print(classification_report(y_true= Y_test_cv, y_pred=y_pred_cv))

              precision    recall  f1-score   support

           D       0.95      0.96      0.95       130
           E       0.93      0.90      0.91        97
           F       0.89      0.89      0.89       117
           G       0.88      0.91      0.89       129
           H       0.86      0.88      0.87        94
           I       0.95      0.87      0.91        84
           J       0.96      0.92      0.94        77
           K       0.89      0.96      0.92        56
           L       1.00      0.90      0.95        10
           M       0.80      0.80      0.80         5

    accuracy                           0.91       799
   macro avg       0.91      0.90      0.90       799
weighted avg       0.91      0.91      0.91       799



In [398]:
et.fit(X_train_cv, Y_train_cv)

ExtraTreesClassifier(bootstrap=True,
                     class_weight={'D': 0.6171817058096415,
                                   'E': 0.823927392739274,
                                   'F': 0.682103825136612,
                                   'G': 0.6187112763320942,
                                   'H': 0.8491496598639455,
                                   'I': 0.9565134099616859,
                                   'J': 1.0358921161825727,
                                   'K': 1.4265714285714286, 'L': 7.8015625,
                                   'M': 15.13030303030303},
                     criterion='entropy', max_depth=15, n_estimators=1000,
                     warm_start=True)

In [399]:
# Training accuracy
y_pred_cv = et.predict(X_train_cv)
print(classification_report(y_true= Y_train_cv, y_pred=y_pred_cv))

              precision    recall  f1-score   support

           D       0.95      0.97      0.96       517
           E       0.94      0.89      0.91       388
           F       0.90      0.88      0.89       469
           G       0.88      0.89      0.88       516
           H       0.86      0.90      0.88       376
           I       0.88      0.88      0.88       334
           J       0.90      0.88      0.89       309
           K       0.87      0.96      0.91       224
           L       1.00      0.78      0.88        41
           M       0.95      0.86      0.90        21

    accuracy                           0.90      3195
   macro avg       0.91      0.89      0.90      3195
weighted avg       0.90      0.90      0.90      3195



In [400]:
#  testing Accureacy
y_pred_cv = et.predict(X_test_cv)
print(classification_report(y_true= Y_test_cv, y_pred=y_pred_cv))

              precision    recall  f1-score   support

           D       0.97      0.98      0.98       130
           E       0.94      0.94      0.94        97
           F       0.92      0.89      0.90       117
           G       0.89      0.92      0.90       129
           H       0.91      0.83      0.87        94
           I       0.87      0.89      0.88        84
           J       0.90      0.90      0.90        77
           K       0.86      0.91      0.89        56
           L       0.90      0.90      0.90        10
           M       1.00      1.00      1.00         5

    accuracy                           0.91       799
   macro avg       0.92      0.92      0.92       799
weighted avg       0.91      0.91      0.91       799



In [288]:
# skf = StratifiedKFold(n_splits=5, shuffle=True)

#### Prediction on X_tets

In [407]:
y_test_pred = et.predict(X_test)

In [408]:
print(classification_report(y_pred=y_test_pred, y_true=Y_test))

              precision    recall  f1-score   support

           D       0.74      0.84      0.78       162
           E       0.46      0.40      0.43       121
           F       0.48      0.50      0.49       146
           G       0.43      0.40      0.41       162
           H       0.36      0.36      0.36       118
           I       0.29      0.28      0.28       104
           J       0.48      0.50      0.49        96
           K       0.57      0.69      0.62        70
           L       0.00      0.00      0.00        13
           M       0.00      0.00      0.00         7

    accuracy                           0.49       999
   macro avg       0.38      0.40      0.39       999
weighted avg       0.47      0.49      0.48       999



### ExtraTreesClassifier

#### Stratified fold

In [157]:
et_skf = ExtraTreesClassifier( criterion = 'entropy',
                            class_weight=balanced, 
                            warm_start=True, 
                            max_samples=None, 
                            bootstrap=True,
                            max_depth=12,
                            n_estimators=1000
                            )

In [160]:
lst_accu_stratified = []
lst_accu_stratified_train = []
skf = StratifiedKFold(n_splits=5)

for train_index, test_index in skf.split(X_train, Y_train):
  x_train_fold, x_test_fold = X.iloc[train_index], X.iloc[test_index]
  y_train_fold, y_test_fold = Y[train_index], Y[test_index]

  clf_ = et_skf

  clf_.fit(x_train_fold, y_train_fold)
  y_train_pred = clf_.predict(x_train_fold)
  Y_pred = clf_.predict(x_test_fold)
  print(classification_report(y_test_fold, Y_pred))

  lst_accu_stratified.append(metrics.f1_score(y_test_fold,Y_pred,average='weighted'))
  lst_accu_stratified_train.append(clf_.score(x_train_fold,y_train_fold))

print(f"train_accuracy={lst_accu_stratified_train}")
print(f"f1_score={lst_accu_stratified}")

print(f" over all train_accuracy={np.mean(lst_accu_stratified_train)*100}")
print(f"f1_score_mean = {np.mean(lst_accu_stratified)*100}")

              precision    recall  f1-score   support

           D       0.86      0.78      0.81       116
           E       0.47      0.53      0.50        83
           F       0.42      0.39      0.40       122
           G       0.44      0.44      0.44       135
           H       0.29      0.31      0.30        90
           I       0.33      0.35      0.34        86
           J       0.39      0.47      0.43        80
           K       0.51      0.55      0.53        65
           L       0.00      0.00      0.00        14
           M       1.00      0.12      0.22         8

    accuracy                           0.47       799
   macro avg       0.47      0.39      0.40       799
weighted avg       0.47      0.47      0.46       799

              precision    recall  f1-score   support

           D       0.98      0.99      0.98       126
           E       0.97      0.98      0.97       117
           F       0.99      0.95      0.97       107
           G       0.96 

In [161]:
scores = cross_val_score(et_skf, X_train, Y_train, scoring='f1_weighted', cv=5)
scores

array([0.50194083, 0.51969182, 0.50525516, 0.50348896, 0.50184247])

In [164]:
y_preds  = et_skf.predict(X_test)

In [165]:
print(classification_report(Y_test, y_preds))

              precision    recall  f1-score   support

           D       0.92      0.90      0.91       162
           E       0.77      0.79      0.78       121
           F       0.80      0.75      0.77       146
           G       0.76      0.77      0.77       162
           H       0.76      0.80      0.78       118
           I       0.71      0.72      0.72       104
           J       0.71      0.74      0.72        96
           K       0.74      0.84      0.79        70
           L       0.83      0.38      0.53        13
           M       1.00      0.29      0.44         7

    accuracy                           0.78       999
   macro avg       0.80      0.70      0.72       999
weighted avg       0.79      0.78      0.78       999



In [177]:
joblib.dump(
        clf_,  
        os.path.join('/home/pankaj_v/Documents/Data_analysis_dixit/Models', f"f1_{round(np.mean(lst_accu_stratified)*100,2)}_2_08.pkl") 
    )

['/home/pankaj_v/Documents/Data_analysis_dixit/Models/f1_89.69_2_08.pkl']

In [ ]:
joblib.dump(
        clf_,  
        os.path.join('/home/pankaj_v/Documents/Data_analysis_dixit/Models', f"f1_{round(np.mean(lst_accu_stratified)*100,2)}_2_08.pkl") 
    )

['/home/pankaj_v/Documents/Data_analysis_dixit/Models/f1_89.69_2_08.pkl']